In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# 1. Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Spark NLP Clustering") \
    .master("spark://172.18.0.3:7077") \
    .config("spark.driver.memory", "8g") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.0") \
    .getOrCreate()

# 2. Đọc file JSONL
input_path = "/opt/workspace/gen_1604_formated.jsonl"
df = spark.read.json(input_path, multiLine=False)

# 3. Tách từng message
df_exploded = df.select(explode("messages").alias("msg"))

# 4. Lọc các câu có role là "user"
user_msgs = df_exploded.filter(col("msg.role") == "user") \
    .select(col("msg.content").alias("user_content"))

# 5. Đếm số lần xuất hiện
content_count = user_msgs.groupBy("user_content") \
    .count() \
    .orderBy(col("count").desc())

# 6. Thêm cột số thứ tự (index)
window_spec = Window.orderBy(col("count").desc())
content_with_index = content_count.withColumn("stt", row_number().over(window_spec))

# 7. Hiển thị đầy đủ nội dung và có số thứ tự
content_with_index.select("stt", "user_content", "count") \
    .show(n=1000, truncate=False)
